In [ ]:
import pandas as pd
import datetime
import pymysql

In [ ]:
def dbcon():
    connection = pymysql.connect(host='localhost',
                             user='pydb01',
                             password='jhfajl8708795',
                             database='python_db_01',
                             cursorclass=pymysql.cursors.DictCursor,
                            autocommit=True)
    connection = connection.cursor()   
    return connection       

In [ ]:
connection=dbcon()


In [ ]:
def hash_gen(num,b,numerals="0123456789abcdefghijklmnopqrstuvwxyz"):
    return ((num == 0) and numerals[0]) or (hash_gen(num // b, b, numerals).lstrip(numerals[0]) + numerals[num % b])

def surr_gen(key_id):
    return (key_id*1679979167)%(36**6)

def get_gen_seq(itcrsr, tbn):
    
    flag = True
    
    while flag:
        sql_insert_statement = 'INSERT INTO gen_seq (surr_key, key_hash) values (null,null)'
        itcrsr.execute(sql_insert_statement)
        
        key_id = itcrsr.lastrowid
        surr_key = surr_gen(key_id)
        hash_key = hash_gen(surr_key,36)
        
        sql_select_statement = "SELECT * FROM gen_seq WHERE surr_key='%s'" %(surr_key)
        itcrsr.execute(sql_select_statement)
        result = itcrsr.fetchall()
        
        if len(result) != 0:
            pass
        else:
            flag = False
            sql_update_statement = """
            UPDATE gen_seq
            SET surr_key=%s,
            key_hash='%s'
            WHERE
            key_id=%s""" %(surr_key,hash_key,key_id)
            itcrsr.execute(sql_update_statement)
            #itcrsr.commit()
        return surr_key
    return newly_generated_id

In [ ]:
inp={
"f_nm":"Iron",
    "l_nm":"Man",
    "dob":"",
    "g":"m",
    "eml":"ironman@avengers.com",
    "img":"",
    "ph":"1765490324",
    "al1":"",
    "al2":"",
    "al3":"",
    "tg":"6768298378"
}


In [ ]:
class cust:
    def __init__ (self, connection, name=None, phone=None, email=None):
        self.name=name
        self.phone=phone
        self.email=email
        self.connection=connection
        
    def check_access(self,inp,connection):
        '''
            Function to check if an email is present in system or not
            if not generate a new cust id and store email
            store email, phone, address etc
            return cust id
        '''

        eml=inp['eml']
        check_qry="""
            select
                cust_id,
                eml
            from
                email
            where
                eml='%s' and
                e_dt='9999-12-31'
        """ %(eml)
        try:
            connection.execute(check_qry)
            cntres=connection.fetchall()
            print (cntres)

            if len(cntres)== 1:
                ### user exists
                retdct={
                    "data":{
                        "idc":str(cntres[0]['cust_id'])
                        },
                    "error":[]            
                }
            elif len(cntres)== 0:
                #### user doesnt exist
                #### need to create one
                '''
                    generate new unique cust id
                    store in cust table
                    store in email table
                    store in phone table
                    store in tg table
                    store in address table   
                    return newly generated cust id
                '''
                new_cust_id=get_gen_seq(connection,"cust_py")
                create_tm=datetime.datetime.today()

                ins_cust_sql="""
                   insert into cust_py(cust_id,st_dt,first_name,last_name,dob,g,e_dt,image_path) 
                   values (%s, '%s', '%s', '%s', '%s', '%s', '%s', '%s') 
                """ %(new_cust_id, create_tm, inp['f_nm'], inp['l_nm'], 
                      inp['dob'], inp['g'],'9999-12-31',inp['img'])
                connection.execute(ins_cust_sql)


                #### code to execute ins_cust_sql

                ins_email_sql="""
                   insert into email (cust_id,st_dt,eml,e_dt) 
                   values (%s, '%s', '%s', '%s') 
                """ %(new_cust_id, create_tm, inp['eml'],'9999-12-31')
                connection.execute(ins_email_sql)


                #### code to execute ins_email_sql

                ins_phone_sql="""
                   insert into phone (cust_id,st_dt,phone,e_dt) 
                   values (%s, '%s', '%s', '%s') 
                """ %(new_cust_id, create_tm, inp['ph'],'9999-12-31')
                connection.execute(ins_phone_sql)
                #### code to execute ins_phone_sql

                ins_tg_sql="""
                   insert into tg (cust_id,st_dt,tg_chat_id,e_dt) 
                   values (%s, '%s', '%s', '%s') 
                """ %(new_cust_id, create_tm, inp['tg'],'9999-12-31')
                connection.execute(ins_tg_sql)

                #### code to execute ins_tg_sql

                ins_add_sql="""
                   insert into address (cust_id,st_dt,al1, al2, al3,e_dt) 
                   values (%s, '%s', '%s', '%s','%s','%s') 
                """ %(new_cust_id, create_tm, inp['al1'], inp['al2'], inp['al3'],'9999-12-31')
                connection.execute(ins_add_sql)

                #### code to execute ins_al_sql

                cnctn.commit()

                retdct={
                    "data":{
                        "idc":str(new_cust_id)
                        },
                    "error":[]            
                }

            elif len(cntres)> 1:
                #### we should handle this
                #### this can only happen when there is an issue in the database
                #### this is an error and an exception must be raised
                connection.rollback(),m
                raise ValueError("Duplicate email addresses in database")

        except ValueError as ex:

            ins_err_log="""
                insert into err_log (er_dt, er_msg, er_comp) values ('%s', '%s', '%s')
            """ %(datetime.datetime.today(), ex, 'check_access')
            connection.execute(ins_err_log)

            retdct={
                    "data":{},
                    "error":[700, "Something went wrong. Please try in some time"]
                }


        except Exception as ex:

            ins_err_log="""
                insert into err_log (er_dt, er_msg, er_comp) values ('%s', '%s', '%s')
            """ %(datetime.datetime.today(), ex, 'check_access')
            connection.execute(ins_err_log)

            retdct={
                    "data":{},
                    "error":[700, "Something went wrong. Please try in some time"]
                }

        finally:
            #### create access log if needed
            #connection.commit()        
            return retdct        


In [ ]:
c1=cust(connection)

In [ ]:
c1.check_access(inp,connection)